In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [6]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [7]:
y_train

,diagnosis
69,B
283,M
560,B
430,M
8,M
...,...
239,M
15,M
335,M
497,B


In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [28]:
X_train_tensor=torch.from_numpy(X_train).float()
X_test_tensor=torch.from_numpy(X_test).float()
y_train_tensor=torch.from_numpy(y_train).float()
y_test_tensor=torch.from_numpy(y_test).float()

In [29]:
X_train_tensor.shape
X_train_tensor.dtype

torch.float32

In [30]:
# import torch.nn as nn
# class MySimpleNN(nn.Module):

#   def __init__(self, num_features):
#     super().__init__()
#     self.linear=nn.Linear(num_features,1)
#     self.sigmoid=nn.Sigmoid()

#   def forward(self, features):
#     out=self.linear(features)
#     out=self.sigmoid(out)
#     return out

#   def loss_function(self, y_pred, y):
#     # Clamp predictions to avoid log(0)
#     epsilon = 1e-7
#     y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

#     # Calculate loss
#     loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
#     return loss



import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [31]:
learning_rate = 0.1
epochs = 25

In [32]:
# define loss function
# binary cross entropy loss function
loss_function = nn.BCELoss()

In [33]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.8862884044647217
Epoch: 2, Loss: 0.6561826467514038
Epoch: 3, Loss: 0.5240517854690552
Epoch: 4, Loss: 0.44588473439216614
Epoch: 5, Loss: 0.3952678442001343
Epoch: 6, Loss: 0.35930120944976807
Epoch: 7, Loss: 0.3320213854312897
Epoch: 8, Loss: 0.3103877604007721
Epoch: 9, Loss: 0.29267919063568115
Epoch: 10, Loss: 0.27783769369125366
Epoch: 11, Loss: 0.26517024636268616
Epoch: 12, Loss: 0.2542002499103546
Epoch: 13, Loss: 0.2445867359638214
Epoch: 14, Loss: 0.23607808351516724
Epoch: 15, Loss: 0.22848352789878845
Epoch: 16, Loss: 0.22165538370609283
Epoch: 17, Loss: 0.2154771238565445
Epoch: 18, Loss: 0.20985530316829681
Epoch: 19, Loss: 0.20471405982971191
Epoch: 20, Loss: 0.19999104738235474
Epoch: 21, Loss: 0.1956344097852707
Epoch: 22, Loss: 0.1916007399559021
Epoch: 23, Loss: 0.18785341084003448
Epoch: 24, Loss: 0.18436110019683838
Epoch: 25, Loss: 0.1810970902442932


In [34]:
model.linear.weight

Parameter containing:
tensor([[ 0.3045,  0.1706,  0.1618,  0.1820,  0.1592,  0.2618,  0.2944,  0.1126,
          0.1751,  0.0224,  0.0578, -0.1268,  0.1749,  0.0662,  0.0632, -0.0387,
         -0.0198,  0.0393, -0.0017, -0.1490,  0.3803,  0.2486,  0.3792,  0.1321,
          0.1268,  0.2388,  0.2290,  0.1801,  0.2011, -0.1124]],
       requires_grad=True)

In [35]:
with torch.no_grad():
    y_pred = model(X_test_tensor)          # get predictions
    y_pred = (y_pred > 0.5).float()        # convert to 0/1
    correct = (y_pred == y_test_tensor).float().sum()   # count correct
    total = y_test_tensor.shape[0]         # total samples
    accuracy = correct / total             # accuracy
    print("Accuracy:", accuracy.item())


Accuracy: 58.92982482910156
